# EECS 461 Machine Learning Assignment 1
### 05 Dec. 2017

### Amer Nour Eddin | 213171245

Run all cells until you reach the main function pass you required (datapath, test_ration, labelcolumn) to it
and it will give you the prepared data. At the end you can evaluate your model and predict values for your label column using 
predict function.
I added tow error mesrueing metrics at the very end of the code as an extra to see how is your model performing.

###### PS. I tested my code on 2 other datasets and it performed well given the assignment requirments. (It can be enhanced, but I didn't want to go out the scope of the assignment)

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals
import numpy as np
import pandas as pd
import os

# Data Analysis & Pre-processing

In Data Analysis & Pre-processing part, you are going to implement some functions using the scikit-learn library. You will
analyze your dataset and change some values in it in order to prepare your data to be used in machine learning models.

#### (a) 
The create_df function should take the path of the csv file and turn it into a pandas data frame.

In [2]:
#(a)

def create_df(data_path):    
    # input: the path of the csv file
    # output: data frame
    df = pd.read_csv(data_path)
    return df

In [3]:
path = os.path.realpath("housing.csv")
df = create_df(path)
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


#### (b) 
The nan_columns function should take a data frame as an input and return a list of names of columns that contain
nan values in it.

In [4]:
#(b)

def nan_columns(df):
    # input: data frame
    # output: a list of names of columns that contain nan values in the data frame
    nancolumns = df.columns[df.isnull().any()].tolist() # take df and list the columns names in which there are NAN values in it

    return nancolumns

In [5]:
nancolumns = nan_columns(df)
nancolumns

['total_bedrooms']

#### (c) 
The categorical_columns function should take a data frame as an input and return a list of column names that contain
categorical values in it. 
For example, if a data frame has ‘gender’ column and ‘female’ and ‘male’ values in it, then ‘gender’ column should
be in the returned list.

In [6]:
#(c)

def categorical_columns(df):
    # input: data frame
    # output: a list of column names that contain categorical values in the data frame
    all_c = df.columns # all columns
    num_cols = df._get_numeric_data().columns # only numerical columns
    catcolumns = list(set(all_c) - set(num_cols)) # subtract all columns from numerical columns => categorical columns(non-numeric)
    return catcolumns

In [7]:
catcolumns = categorical_columns(df)
catcolumns

['ocean_proximity']

#### (d) 
The replace_missing_features function should take a data frame and a list of column names that contain nan values
as input.The function should replace all nan values in the column with the median of this column’s values and return this
new data frame as output.

In [8]:
#(d)

def replace_missing_features(df, nancolumns):
    # input: data frame, list of column names that contain nan values
    # output: data frame
    if len(nancolumns) == 0:
        new_df1 = df
    else:
        for c in nancolumns:
            new_df1 = df.fillna((df[c]).median()) # fill all NAN elements with the median of the column elements
    return new_df1
    

In [9]:
nancolumns = nan_columns(df)
new_df1 = replace_missing_features(df, nancolumns)
# new_df1

#### (e) 
The cat_to_num function should take a data frame and a list of categorical feature column names as input. Perform
one-hot-encoding on categorical features. Modify your data frame and replace its nominal features with their one-hot
encoding representation.

In [10]:
# I WILL USE THIS CATAGORICAL ENCODER IN PART (e) cat_to_num function so you have to compile it first!



# # Definition of the CategoricalEncoder class, copied from PR #9151. # #

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from scipy import sparse

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    """Encode categorical features as a numeric array.
    The input to this transformer should be a matrix of integers or strings,
    denoting the values taken on by categorical (discrete) features.
    The features can be encoded using a one-hot aka one-of-K scheme
    (``encoding='onehot'``, the default) or converted to ordinal integers
    (``encoding='ordinal'``).
    This encoding is needed for feeding categorical data to many scikit-learn
    estimators, notably linear models and SVMs with the standard kernels.
    Read more in the :ref:`User Guide <preprocessing_categorical_features>`.
    Parameters
    ----------
    encoding : str, 'onehot', 'onehot-dense' or 'ordinal'
        The type of encoding to use (default is 'onehot'):
        - 'onehot': encode the features using a one-hot aka one-of-K scheme
          (or also called 'dummy' encoding). This creates a binary column for
          each category and returns a sparse matrix.
        - 'onehot-dense': the same as 'onehot' but returns a dense array
          instead of a sparse matrix.
        - 'ordinal': encode the features as ordinal integers. This results in
          a single column of integers (0 to n_categories - 1) per feature.
    categories : 'auto' or a list of lists/arrays of values.
        Categories (unique values) per feature:
        - 'auto' : Determine categories automatically from the training data.
        - list : ``categories[i]`` holds the categories expected in the ith
          column. The passed categories are sorted before encoding the data
          (used categories can be found in the ``categories_`` attribute).
    dtype : number type, default np.float64
        Desired dtype of output.
    handle_unknown : 'error' (default) or 'ignore'
        Whether to raise an error or ignore if a unknown categorical feature is
        present during transform (default is to raise). When this is parameter
        is set to 'ignore' and an unknown category is encountered during
        transform, the resulting one-hot encoded columns for this feature
        will be all zeros.
        Ignoring unknown categories is not supported for
        ``encoding='ordinal'``.
    Attributes
    ----------
    categories_ : list of arrays
        The categories of each feature determined during fitting. When
        categories were specified manually, this holds the sorted categories
        (in order corresponding with output of `transform`).
    Examples
    --------
    Given a dataset with three features and two samples, we let the encoder
    find the maximum value per feature and transform the data to a binary
    one-hot encoding.
    >>> from sklearn.preprocessing import CategoricalEncoder
    >>> enc = CategoricalEncoder(handle_unknown='ignore')
    >>> enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])
    ... # doctest: +ELLIPSIS
    CategoricalEncoder(categories='auto', dtype=<... 'numpy.float64'>,
              encoding='onehot', handle_unknown='ignore')
    >>> enc.transform([[0, 1, 1], [1, 0, 4]]).toarray()
    array([[ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
           [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])
    See also
    --------
    sklearn.preprocessing.OneHotEncoder : performs a one-hot encoding of
      integer ordinal features. The ``OneHotEncoder assumes`` that input
      features take on values in the range ``[0, max(feature)]`` instead of
      using the unique values.
    sklearn.feature_extraction.DictVectorizer : performs a one-hot encoding of
      dictionary items (also handles string-valued features).
    sklearn.feature_extraction.FeatureHasher : performs an approximate one-hot
      encoding of dictionary items or strings.
    """

    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):
        """Fit the CategoricalEncoder to X.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_feature]
            The data to determine the categories of each feature.
        Returns
        -------
        self
        """

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' "
                        "or 'ordinal', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or "
                        "'ignore', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for"
                             " encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1}"
                               " during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        """Transform X using one-hot encoding.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data to encode.
        Returns
        -------
        X_out : sparse matrix or a 2-d array
            Transformed input.
        """
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1}"
                           " during transform".format(diff, i))
                    raise ValueError(msg)
                else:
                    # Set the problematic rows to an acceptable value and
                    # continue `The rows are marked `X_mask` and will be
                    # removed later.
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

In [11]:
#(e)

def cat_to_num(new_df1, catcolumns):
    # input: data frame, list of categorical feature column names
    # output: data frame
    if len(catcolumns) == 0:
        new_df2 = new_df1
    else:
        housing_cat = new_df1[catcolumns] # take all catagorical columns
        cat_encoder = CategoricalEncoder(encoding="onehot-dense") # specify the encoder
        housing_cat_reshaped = housing_cat.values.reshape(-1, 1) # reshap
        dense_m = cat_encoder.fit_transform(housing_cat_reshaped) # encode 1hot
        dense_m_df = pd.DataFrame(dense_m) # convert to dataframe
        c = cat_encoder.categories_ # list of the catagories names
        dense_m_df.columns = c # assign the catagories names to the encoded dataframe
        # add the ready categorical encoded dataframe to the original dataframe and remove the columns that we encoded (non-numeric) 
        new_df2 = pd.concat([new_df1, dense_m_df], axis=1).drop(catcolumns, 1) 
    return new_df2


In [12]:
new_df2 = cat_to_num(new_df1, catcolumns )
new_df2.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0.0,0.0,0.0,1.0,0.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0.0,0.0,0.0,1.0,0.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0.0,0.0,0.0,1.0,0.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0.0,0.0,0.0,1.0,0.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0.0,0.0,0.0,1.0,0.0


#### (f) 
The standardization function should take a data frame and label column as input and scale all columns except the
label column with standardization. Return a new data frame with scaled values. (10 points)
Scikit-Learn provides a transformer called StandardScaler for standardization. The output of the scaler is an
array, you need to convert it dataframe after standardization. Don’t forget to add indexes and columns of the
original dataframe to the new dataframe.

In [13]:
#(f)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer

# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values


def standardization(new_df2, labelcol):
    # input: data frame and name of the label column
    # output: scaled data frame
    x =  list(new_df2.columns) # x is a list of all column names in the dataframe
    x.remove(labelcol) # remove label column from x
    x
    # set the parameters and pipeline 
    pipeline = Pipeline([
        ('selector', DataFrameSelector(x)),
        ('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ]) 
    new_df3 = pd.DataFrame(pipeline.fit_transform(new_df2)) # fit and transform and convert the standardized data to a new dataframe 
    new_df3.columns = x # assigne column names to the new dataframe
    new_df3 = pd.concat([new_df3, new_df2[labelcol]], axis=1) # add the non-standardized label column to the standardized dataframe
    return new_df3


In [14]:
new_df3 = standardization(new_df2,'median_income')
new_df3

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_house_value,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN,median_income
0,-1.327835,1.052548,0.982143,-0.804819,-0.972476,-0.974429,-0.977033,2.129631,-0.891156,-0.681889,-0.015566,2.830742,-0.384466,8.3252
1,-1.322844,1.043185,-0.607019,2.045890,1.357143,0.861439,1.669961,1.314156,-0.891156,-0.681889,-0.015566,2.830742,-0.384466,8.3014
2,-1.332827,1.038503,1.856182,-0.535746,-0.827024,-0.820777,-0.843637,1.258693,-0.891156,-0.681889,-0.015566,2.830742,-0.384466,7.2574
3,-1.337818,1.038503,1.856182,-0.624215,-0.719723,-0.766028,-0.733781,1.165100,-0.891156,-0.681889,-0.015566,2.830742,-0.384466,5.6431
4,-1.337818,1.038503,1.856182,-0.462404,-0.612423,-0.759847,-0.629157,1.172900,-0.891156,-0.681889,-0.015566,2.830742,-0.384466,3.8462
5,-1.337818,1.038503,1.856182,-0.786942,-0.772182,-0.894071,-0.801787,0.544611,-0.891156,-0.681889,-0.015566,2.830742,-0.384466,4.0368
6,-1.337818,1.033821,1.856182,-0.046188,-0.114070,-0.292712,0.037823,0.800259,-0.891156,-0.681889,-0.015566,2.830742,-0.384466,3.6591
7,-1.337818,1.033821,1.856182,0.214634,0.358054,-0.237079,0.385698,0.299362,-0.891156,-0.681889,-0.015566,2.830742,-0.384466,3.1200
8,-1.342809,1.033821,1.061601,-0.037021,0.305595,-0.193810,0.249687,0.171971,-0.891156,-0.681889,-0.015566,2.830742,-0.384466,2.0804
9,-1.337818,1.033821,1.856182,0.418616,0.405743,0.110844,0.560944,0.470083,-0.891156,-0.681889,-0.015566,2.830742,-0.384466,3.6912


#### (g)
The my_train_test_split function should take a data frame, name of the label column and percentage of test size as
input. Split dataframe as X and y where y is the label column values and X is the feature values (all column values
except the label column). Then, the function should split X and y into test and train sets as X_train, X_test, y_train and
y_test with the given test size. Output datatype should be numpy array.
For example, if the test size is 0.3, then the function should divide the data frame into 70% training and 30% test
data.

In [15]:
#(g)

from sklearn.model_selection import train_test_split

def my_train_test_split(new_df3, labelcol, test_ratio):
    # input: data frame, name of the label column and test data percentage
    # output: X_train, X_test, y_train, y_test
    np.random.seed(0) # DON'T ERASE THIS LINE
    y = new_df3[labelcol].values #t he label column values
    X = new_df3.drop(labelcol, 1).values # the feature values (all column values except the label column)
    X_train, X_test = train_test_split(X, test_size=test_ratio, random_state=0) # features Train+Test
    y_train, y_test = train_test_split(y, test_size=test_ratio, random_state=0) # Labels Train+Test
#     X_train, X_test, y_train, y_test = [], [], [], []
    return X_train, X_test, y_train, y_test



In [16]:
# X_train, X_test, y_train, y_test = my_train_test_split(new_df3, 'price', 0.3)

#### (h)
The main function will use all functions you created and return a train and test set at the end. The function should
take path of a csv file, test data percentage and name of the label column as an input. First, convert the csv data (which
you downloaded at the beginning) into a data frame (a). Fill nan columns of this data frame (b). Find in which column
there are categorical values (c). Fill up missing features in data frames (d). Convert categorical features into numerical
format (e). Scale all feature columns with standardization (f). Split the final data frame into train and test matrices
according to given label column and test ratio. Return X_train, X_test, y_train and y_test matrices (g).

## main()

In [17]:
#(h)

def main(dataPath, testRatio, labelColumn):
    # input: the path of the csv file, test data percentage and name of the label column
    # output: X_train, X_test, y_train, y_test as numpy arrays
    df = create_df(dataPath) #(a)
    nancolumns = nan_columns(df) #(b)
    catcolumns = categorical_columns(df) #(c)
    new_df1 = replace_missing_features(df, nancolumns) #(d)
    new_df2 = cat_to_num(new_df1, catcolumns ) #(e)
    new_df3 = standardization(new_df2, labelColumn) #(f)    
    X_train, X_test, y_train, y_test = my_train_test_split(new_df3, labelColumn, testRatio) #(g)
    return X_train, X_test, y_train, y_test


In [32]:
path = os.path.realpath("housing.csv")
X_train, X_test, y_train, y_test = main(path, 0.3, 'median_income') #you can change the parameters for main as you want
X_train, X_test, y_train, y_test


(array([[-0.10995627,  0.51413373,  1.85618152, ..., -0.01556621,
         -0.35326426, -0.38446649],
        [-1.31785261,  1.00104776,  1.14105882, ..., -0.01556621,
          2.83074203, -0.38446649],
        [ 0.76352248, -0.82487986, -0.92485123, ..., -0.01556621,
         -0.35326426, -0.38446649],
        ..., 
        [-1.16312209,  0.44390574,  0.18756187, ..., -0.01556621,
         -0.35326426,  2.60100692],
        [ 0.81842686, -0.9419265 ,  0.42593611, ..., -0.01556621,
         -0.35326426, -0.38446649],
        [ 2.00136666, -1.32583949, -1.08376738, ..., -0.01556621,
         -0.35326426, -0.38446649]]),
 array([[ 1.25766189, -1.42884054, -0.52756083, ..., -0.01556621,
         -0.35326426,  2.60100692],
        [ 0.79846163, -0.80147053,  0.26701995, ..., -0.01556621,
         -0.35326426, -0.38446649],
        [-1.13317424,  1.41305194,  0.02864572, ..., -0.01556621,
         -0.35326426, -0.38446649],
        ..., 
        [ 0.82840948, -0.81551613,  0.10810379, ...,

# Model Evaluation

In this section, you are going to train a Linear Regression model that learns to predict the label column according to rest
of the features in our dataset.


#### (i)
model_evaluation function should take training instances (train_x) and labels (train_y) as numpy arrays. Create a
LinearRegression model with default parameters. Train the model with the train_x and train_y. Your function should
return coef_ and intercept_ arrays of the model as output.

In [33]:
#(i)

from sklearn.linear_model import LinearRegression

def model_evaluation(X_train, y_train):
    # input: X_train and y_train matrices
    # output: LinearRegression model's coef_ and intercept_ parameters
    lin_reg = LinearRegression() # define the Linear Regression model
    lin_reg.fit(X_train, y_train) # train it
    coef_, intercept_ = [lin_reg.coef_], [lin_reg.intercept_] # get the coeffecient and the intercept values for your defined model
    return coef_, intercept_


In [34]:
coef_, intercept_ = model_evaluation(X_train, y_train)
coef_, intercept_

([array([  1.53561628e-02,  -3.00066223e-02,  -2.74086746e-01,
           1.93936473e+00,  -1.86574215e+00,   5.08633026e-02,
          -1.43725983e-01,   1.17283330e+00,   1.65186942e+13,
           1.54798631e+13,   5.17559722e+11,   1.04450136e+13,
           1.11396031e+13])], [3.8655306274684427])

#### (j) 
predict function should take instance matrix, coef_ array and intercept_ array as input. This function will create a Linear Regression model and set its coef_ and intercept_ parameters to input coef_ and intercept_ values. Using this model, predict the label column values of given instance/s. Return list of predictions as output.

In [35]:
#(j)

def predict(instance, coef_, intercept_):
    # input: instance matrix, coef_ array and intercept_ array
    # ouput: list of predictions for input instances
    lin_reg2 = LinearRegression() # define a Linear Regression model
    coeff = np.copy(coef_) # coeff holds the coeffecient values in the input
    intercept = np.copy(intercept_) # intercept holds the intercept value in the input
    lin_reg2.coef_= coeff # set coeff as the coeffecient for your defined model
    lin_reg2.intercept_ = intercept # set intercept as the intercept for your defined model
    predictions = lin_reg2.predict(instance) # predict
    return predictions



In [40]:
predict(X_test[:1], coef_, intercept_) # Predict first 10 instances from X_test

array([[ 3.32256188]])

In [37]:
y_test[:10] # Labels

array([ 4.1518,  5.7796,  4.3487,  2.4511,  5.0049,  2.5682,  6.0661,
        5.9275,  6.3373,  3.9167])

### Mesureing the error value for this Linear Regression model

In [24]:
from sklearn.metrics import mean_squared_error

predictions = predict(X_test, coef_, intercept_)
lin_mse = mean_squared_error(y_test, predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

1.1547043568210507

In [25]:
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(y_test, predictions)
lin_mae

0.76544182202657429